In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

# Download data

In [2]:
df = pd.read_csv('Trips_2018.csv',index_col=0)

# Look at the raw data

In [3]:
print(df.head())
print(df.info())
print(df.describe())

   tripduration                 starttime                  stoptime  \
0           970  2018-01-01 13:50:57.4340  2018-01-01 14:07:08.1860   
1           723  2018-01-01 15:33:30.1820  2018-01-01 15:45:33.3410   
2           496  2018-01-01 15:39:18.3370  2018-01-01 15:47:35.1720   
3           306  2018-01-01 15:40:13.3720  2018-01-01 15:45:20.1910   
4           306  2018-01-01 18:14:51.5680  2018-01-01 18:19:57.6420   

   start_station_id  start_station_latitude  start_station_longitude  \
0              72.0               40.767272               -73.993929   
1              72.0               40.767272               -73.993929   
2              72.0               40.767272               -73.993929   
3              72.0               40.767272               -73.993929   
4              72.0               40.767272               -73.993929   

   end_station_id  end_station_latitude  end_station_longitude  bikeid  \
0           505.0             40.749013             -73.988484   3

## Initial Data Exploration

In [4]:
# Check data shape and info
print(f"Dataset shape: {df.shape}")
print("\nData types and missing values:")
print(df.info())
print("\nBasic statistics:")
print(df.describe())
print("\nMissing values:")
print(df.isnull().sum())

Dataset shape: (17548339, 13)

Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
Index: 17548339 entries, 0 to 17548338
Data columns (total 13 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start_station_id         float64
 4   start_station_latitude   float64
 5   start_station_longitude  float64
 6   end_station_id           float64
 7   end_station_latitude     float64
 8   end_station_longitude    float64
 9   bikeid                   int64  
 10  usertype                 object 
 11  birth_year               int64  
 12  gender                   int64  
dtypes: float64(6), int64(4), object(3)
memory usage: 1.8+ GB
None

Basic statistics:
       tripduration  start_station_id  start_station_latitude  \
count  1.754834e+07      1.754584e+07            1.754834e+07   
mean   9.887432e+02      1.589282e+03     

## Data Quality Check

In [5]:
# Check for extreme outliers in trip duration
print("Trip duration statistics:")
print(f"Max trip duration: {df['tripduration'].max()} seconds ({df['tripduration'].max()/3600:.1f} hours)")
print(f"Min trip duration: {df['tripduration'].min()} seconds")

# Check birth year range
print("\nBirth year range:")
print(f"Min birth year: {df['birth_year'].min()}")
print(f"Max birth year: {df['birth_year'].max()}")



Trip duration statistics:
Max trip duration: 19510049 seconds (5419.5 hours)
Min trip duration: 61 seconds

Birth year range:
Min birth year: 1885
Max birth year: 2002


# Removel of Missing Station IDs (can't assign to clusters)

In [6]:
# Remove rows with missing station IDs (can't assign to clusters)
print(f"Rows before dropping missing stations: {len(df)}")
df = df.dropna(subset=['start_station_id', 'end_station_id'])
print(f"Rows after dropping missing stations: {len(df)}")
print(f"Rows removed: {17548339 - len(df)}")

Rows before dropping missing stations: 17548339
Rows after dropping missing stations: 17545842
Rows removed: 2497


# Handle Outliers

In [7]:
# Filter unrealistic trip durations (remove trips > 24 hours)
max_duration = 24 * 3600  # 24 hours in seconds
print(f"\nRows before trip duration filter: {len(df)}")
df = df[df['tripduration'] <= max_duration]
print(f"Rows after trip duration filter: {len(df)}")

# Filter unrealistic birth years (assume riders are between 16 and 90 years old in 2018)
df = df[(df['birth_year'] >= 1928) & (df['birth_year'] <= 2002)]
print(f"Rows after birth year filter: {len(df)}")


Rows before trip duration filter: 17545842
Rows after trip duration filter: 17541330
Rows after birth year filter: 17527883


# Optimize Memory

In [8]:
# Optimize data types to reduce memory usage
print(f"Memory usage before optimization: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB")

df['gender'] = df['gender'].astype('int8')
df['bikeid'] = df['bikeid'].astype('int32')
df['birth_year'] = df['birth_year'].astype('int16')
df['usertype'] = df['usertype'].astype('category')
df['start_station_id'] = df['start_station_id'].astype('int16')
df['end_station_id'] = df['end_station_id'].astype('int16')

print(f"Memory usage after optimization: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB")

Memory usage before optimization: 5.13 GB
Memory usage after optimization: 3.61 GB


# Convert Datetime

In [9]:
# Convert to datetime and extract time features
df['starttime'] = pd.to_datetime(df['starttime'])
df['stoptime'] = pd.to_datetime(df['stoptime'])

# Verify tripduration matches actual time difference
df['calculated_duration'] = (df['stoptime'] - df['starttime']).dt.total_seconds()
duration_mismatch = abs(df['tripduration'] - df['calculated_duration']) > 60  # Allow 60 sec tolerance
print(f"Rows with duration mismatch: {duration_mismatch.sum()}")

# Remove problematic rows (very small fraction of the data)
df = df[~duration_mismatch]
print(f"Removed {duration_mismatch.sum()} rows with duration mismatch")

# Extract time features
df['date'] = df['starttime'].dt.date
df['hour'] = df['starttime'].dt.hour
df['day_of_week'] = df['starttime'].dt.day_name()
df['month'] = df['starttime'].dt.month
df['week'] = df['starttime'].dt.isocalendar().week

print("\nDatetime conversion complete")
print(df[['starttime', 'stoptime', 'date', 'hour', 'day_of_week', 'month', 'week']].head())



Rows with duration mismatch: 63
Removed 63 rows with duration mismatch

Datetime conversion complete
                starttime                stoptime        date  hour  \
0 2018-01-01 13:50:57.434 2018-01-01 14:07:08.186  2018-01-01    13   
1 2018-01-01 15:33:30.182 2018-01-01 15:45:33.341  2018-01-01    15   
2 2018-01-01 15:39:18.337 2018-01-01 15:47:35.172  2018-01-01    15   
3 2018-01-01 15:40:13.372 2018-01-01 15:45:20.191  2018-01-01    15   
4 2018-01-01 18:14:51.568 2018-01-01 18:19:57.642  2018-01-01    18   

  day_of_week  month  week  
0      Monday      1     1  
1      Monday      1     1  
2      Monday      1     1  
3      Monday      1     1  
4      Monday      1     1  


# Geographic outliers 

In [10]:
# Check how many rows will be affected by geographic filter
print("Geographic outlier analysis:")
print(f"Total rows before filter: {len(df)}")

start_outliers = df['start_station_latitude'] > 45
end_outliers = df['end_station_latitude'] > 45
any_outliers = start_outliers | end_outliers

print(f"Rows with start_station_latitude > 45: {start_outliers.sum()}")
print(f"Rows with end_station_latitude > 45: {end_outliers.sum()}")
print(f"Total rows with ANY station > 45°: {any_outliers.sum()}")
print(f"Percentage of data: {(any_outliers.sum() / len(df)) * 100:.4f}%")

# Apply the filter
df = df[(df['start_station_latitude'] <= 45) & 
        (df['end_station_latitude'] <= 45)]

print(f"\nRows after geographic filter: {len(df)}")
print(f"Rows removed: {any_outliers.sum()}")

Geographic outlier analysis:
Total rows before filter: 17527820
Rows with start_station_latitude > 45: 79
Rows with end_station_latitude > 45: 79
Total rows with ANY station > 45°: 79
Percentage of data: 0.0005%

Rows after geographic filter: 17527741
Rows removed: 79


# Save the cleaned dataset

In [11]:
print(f"Final shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB")
print(f"\nDate range: {df['starttime'].min()} to {df['starttime'].max()}")
print(f"Number of unique stations: {df['start_station_id'].nunique()}")
print(f"Number of unique bikes: {df['bikeid'].nunique()}")
print(f"\nMissing values per column:")
print(df.isnull().sum())

Final shape: (17527741, 19)
Memory usage: 3.39 GB

Date range: 2018-01-01 00:01:50.650000 to 2018-12-31 23:59:51.085000
Number of unique stations: 816
Number of unique bikes: 15008

Missing values per column:
tripduration               0
starttime                  0
stoptime                   0
start_station_id           0
start_station_latitude     0
start_station_longitude    0
end_station_id             0
end_station_latitude       0
end_station_longitude      0
bikeid                     0
usertype                   0
birth_year                 0
gender                     0
calculated_duration        0
date                       0
hour                       0
day_of_week                0
month                      0
week                       0
dtype: int64


In [12]:
df.to_csv('trips_2018_cleaned.csv', index=False)